<a href="https://colab.research.google.com/github/Aditi0102/ML-notebook/blob/main/AiModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [1]:
%tensorflow_version 2.x

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from IPython.display import clear_output
import cv2
from tensorflow.keras.optimizers import RMSprop
from google.colab.patches import cv2_imshow
import math

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aditiagarwal01","key":"4c56849565c39183e765f285aeafe7d3"}\n'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [5]:
! kaggle datasets download -d mohamedmustafa/real-life-violence-situations-dataset

100% 3.58G/3.58G [01:05<00:00, 48.1MB/s]
100% 3.58G/3.58G [01:05<00:00, 58.6MB/s]


In [6]:
!unzip \*.zip  && rm *.zip
clear_output()
print("Extracted")

Extracted


In [7]:
import os
train_folder = 'Directories/training/violent'
if not os.path.exists(train_folder):
    os.makedirs(train_folder)

In [8]:
# Training Data

count = 0
for i in range (1,800):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (train_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [9]:
import os
test_folder = 'Directories/validation/violent'
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

In [23]:
# Testing Data
count = 0
for i in range (800,1000):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/Violence/V_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Processing Violence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break
        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1
    
    cap.release()
clear_output()
print("Done Training with Violence !!")


Done Training with Violence !!


In [11]:
import os
nv_folder = 'Directories/training/non_violent'
if not os.path.exists(nv_folder):
    os.makedirs(nv_folder)

In [12]:
count = 0
for i in range (1,800):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (nv_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")

Done Training with Non-Violence !!


In [13]:
import os
nv_test_folder = 'Directories/validation/non_violent'
if not os.path.exists(nv_test_folder):
    os.makedirs(nv_test_folder)

In [24]:
count = 0
for i in range (800,1000):
    videoFile = f"/content/real life violence situations/Real Life Violence Dataset/NonViolence/NV_{i}.mp4"
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    print(f"Reading File NonViolence_{i}")
    while True:
        ret, frame = cap.read()
        if ret is False:
            break

        frame = cv2.resize(frame, (64,64))
        fname = '%s/%d.jpg' % (nv_test_folder, count)
        cv2.imwrite(fname, frame)
        count += 1

        count += 1
    cap.release()
clear_output()
print("Done Training with Non-Violence !!")


Done Training with Non-Violence !!


In [15]:
base_dir = './Directories'
train_dir = os.path.join(base_dir,'training')
validation_dir = os.path.join(base_dir,'validation')

In [16]:
train_violent_dir = os.path.join(train_dir,'violent')
train_nonviolent_dir=os.path.join(train_dir,'non_violent')
validation_violent_dir=os.path.join(validation_dir,'violent')
validation_nonviolent_dir=os.path.join(validation_dir,'non_violent')

In [17]:
train_datagen= ImageDataGenerator(rescale=1./255,rotation_range=40, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale =1./255)

In [18]:
train_generator = train_datagen.flow_from_directory(train_dir,color_mode="rgb", target_size = (64,64),batch_size=20, class_mode='binary', shuffle=True)

Found 226270 images belonging to 2 classes.


In [25]:
validation_generator = test_datagen.flow_from_directory(validation_dir,color_mode="rgb",target_size=(64,64),batch_size=20,class_mode='binary', shuffle=True)


Found 54195 images belonging to 2 classes.


In [20]:
model= tf.keras.models.Sequential([
       tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3)),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
       tf.keras.layers.MaxPooling2D(2,2),
       tf.keras.layers.Dropout(0.5),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(512, activation='relu'),
       tf.keras.layers.Dense(1,activation ='sigmoid')

])

In [26]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [27]:
model1= model.fit(train_generator, 
                  steps_per_epoch=100,
                  epochs=40, 
                  validation_data=validation_generator,
                  validation_steps=50,verbose=2)

Epoch 1/40
100/100 - 4s - loss: 0.5803 - accuracy: 0.6635 - val_loss: 0.8932 - val_accuracy: 0.4390
Epoch 2/40
100/100 - 3s - loss: 0.5850 - accuracy: 0.6665 - val_loss: 0.8499 - val_accuracy: 0.4960
Epoch 3/40
100/100 - 3s - loss: 0.5762 - accuracy: 0.6835 - val_loss: 0.7541 - val_accuracy: 0.5160
Epoch 4/40
100/100 - 3s - loss: 0.5603 - accuracy: 0.6980 - val_loss: 0.9308 - val_accuracy: 0.5010
Epoch 5/40
100/100 - 3s - loss: 0.5744 - accuracy: 0.6690 - val_loss: 0.9510 - val_accuracy: 0.5390
Epoch 6/40
100/100 - 3s - loss: 0.5470 - accuracy: 0.6955 - val_loss: 1.2567 - val_accuracy: 0.5450
Epoch 7/40
100/100 - 3s - loss: 0.5499 - accuracy: 0.6900 - val_loss: 1.2234 - val_accuracy: 0.4920
Epoch 8/40
100/100 - 3s - loss: 0.5417 - accuracy: 0.7030 - val_loss: 0.9773 - val_accuracy: 0.5220
Epoch 9/40
100/100 - 3s - loss: 0.5293 - accuracy: 0.7280 - val_loss: 0.8835 - val_accuracy: 0.4840
Epoch 10/40
100/100 - 3s - loss: 0.5478 - accuracy: 0.7060 - val_loss: 0.8550 - val_accuracy: 0.5070

In [28]:
import time
t = time.time()
export_path_keras = "./{}.h5".format(int(t))
print(export_path_keras)

model.save(export_path_keras)

./1611203344.h5
